In [19]:
from collections import defaultdict
from django.db.models.signals import *
import pandas as pd
from itertools import chain
from tqdm.notebook import tqdm

In [234]:
def permissoes_history_user(result_list):
    permissoesUsuarios = {}
    for a in result_list.order_by('history_date'):
        try:
        #print(a.user.get_full_name())
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if not permissoesUsuarios.get(a.user.get_full_name()).get(a.history_date.year): 
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year] = {}
            if a.history_date.month in permissoesUsuarios.get(a.user.get_full_name()).get(a.history_date.year).keys():
                if a.permissions.name == "Inativo":
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.permissions.name+' mesmo mês'] = a.history_date.month
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year]['ultimaAtiva'] = a.permissions.name
                else:
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year]['ultimaAtiva'] = a.permissions.name
            else:
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year]['ultimaAtiva'] = a.permissions.name
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.history_date.month] = a.permissions.name
#             print(permissoesUsuarios)
        except:
            pass

    usuariosAtivos = []
    usuariosInativos = []
    for nome, ano in permissoesUsuarios.items():
        
        item_ano = max(list(ano.keys()))
        item_min_ano = min(list(ano.keys()))
        cargo_atual = ano[item_ano]
        item_mes = max([x for x in cargo_atual.keys() if isinstance(x,int)])
        print(ano,ano[item_ano],cargo_atual[item_mes],cargo_atual[item_mes] != 'Inativo',cargo_atual.get('Inativo mesmo mês') != item_mes,item_min_ano < item_ano)
        if (cargo_atual[item_mes] != 'Inativo' and cargo_atual.get('ultimaAtiva') != 'Inativo') or cargo_atual.get('Inativo mesmo mês') != item_mes or item_min_ano < item_ano:
            usuariosAtivos.append(nome)
        else:
            usuariosInativos.append(nome)

    return usuariosAtivos,usuariosInativos

In [235]:
companies = ['Modera - Consórcio ME2', 
             'Sinalizavia (Municípios)', 
             'Afirma Engenharia Viária',
             'Pavicolet', 
             'Sanson',
             'Entrevias',
             'Televale',
             'Garcia Monteiro',
             'LCM Construção e Comércio',
             'Construtora Castilho']

In [51]:
# Modera, 
#Sinalizavia (Municípios), Afirma Engenharia Viária,Pavicolet (Amapá),Pavicolet (Amazonas), Sanson,Entrevias,Televale,Garcia Monteiro

In [236]:
pd.DataFrame().to_excel('Usuários Ativos Financeiro.xlsx')

In [238]:
dfs=[]
for company in tqdm(companies):

    if 'Pavicolet' in company:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__icontains=company
#             ,history_date__year=year
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        )
    elif company is not 'Entrevias':
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name=company
#             ,history_date__year=year
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by('history_date')
    else:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr'),
            company__name__icontains=company
#             ,history_date__year=year
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    
    users_permissions = permissoes_history_user(historicoUsuarios)[0] if len(permissoes_history_user(historicoUsuarios)[0]) > 0 else 0
    
    if users_permissions != 0:
        df2 = pd.DataFrame(data=users_permissions,columns=['Usuários ativos'])
        dfs.append(df2)
        with pd.ExcelWriter('Usuários Ativos Financeiro.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=company,na_rep="-")

  0%|          | 0/10 [00:00<?, ?it/s]

{2023: {'ultimaAtiva': 'Coordenador', 5: 'Fiscal', 6: 'Engenheiro', 11: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 5: 'Fiscal', 6: 'Engenheiro', 11: 'Coordenador'} Coordenador True True False
{2023: {'ultimaAtiva': 'Coordenador', 5: 'Fiscal', 6: 'Engenheiro', 11: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 5: 'Fiscal', 6: 'Engenheiro', 11: 'Coordenador'} Coordenador True True False
{2023: {'ultimaAtiva': 'Inativo', 5: 'Fiscal', 6: 'Engenheiro', 10: 'Inativo'}} {'ultimaAtiva': 'Inativo', 5: 'Fiscal', 6: 'Engenheiro', 10: 'Inativo'} Inativo False True False
{2023: {'ultimaAtiva': 'Engenheiro', 8: 'Engenheiro'}} {'ultimaAtiva': 'Engenheiro', 8: 'Engenheiro'} Engenheiro True True False
{2023: {'ultimaAtiva': 'Engenheiro', 8: 'Engenheiro', 11: 'Engenheiro'}} {'ultimaAtiva': 'Engenheiro', 8: 'Engenheiro', 11: 'Engenheiro'} Engenheiro True True False
{2023: {'ultimaAtiva': 'Fiscal', 9: 'Fiscal', 10: 'Fiscal'}} {'ultimaAtiva': 'Fiscal', 9: 'Fiscal', 10: 'Fiscal'} Fiscal True True False


{2023: {'ultimaAtiva': 'Administrador', 6: 'Administrador', 10: 'Suporte'}} {'ultimaAtiva': 'Administrador', 6: 'Administrador', 10: 'Suporte'} Suporte True True False
{2023: {'ultimaAtiva': 'Usuário de campo', 6: 'Administrador', 7: 'Inativo', 9: 'Engenheiro', 'Inativo mesmo mês': 9, 10: 'Usuário de campo'}} {'ultimaAtiva': 'Usuário de campo', 6: 'Administrador', 7: 'Inativo', 9: 'Engenheiro', 'Inativo mesmo mês': 9, 10: 'Usuário de campo'} Usuário de campo True True False
{2023: {'ultimaAtiva': 'Suporte', 6: 'Administrador', 9: 'Suporte', 10: 'Administrador', 12: 'Suporte'}} {'ultimaAtiva': 'Suporte', 6: 'Administrador', 9: 'Suporte', 10: 'Administrador', 12: 'Suporte'} Suporte True True False
{2023: {'ultimaAtiva': 'Administrador', 7: 'Administrador', 10: 'Coordenador', 12: 'Administrador'}} {'ultimaAtiva': 'Administrador', 7: 'Administrador', 10: 'Coordenador', 12: 'Administrador'} Administrador True True False
{2023: {'ultimaAtiva': 'Coordenador', 7: 'Administrador', 10: 'Administ

{2022: {'ultimaAtiva': 'Administrador', 1: 'Administrador'}, 2023: {'ultimaAtiva': 'Inativo', 5: 'Inativo', 9: 'Inativo', 10: 'Inativo'}} {'ultimaAtiva': 'Inativo', 5: 'Inativo', 9: 'Inativo', 10: 'Inativo'} Inativo False True True
{2022: {'ultimaAtiva': 'Encarregado', 2: 'Encarregado'}, 2023: {'ultimaAtiva': 'Inativo', 9: 'Inativo', 10: 'Inativo'}} {'ultimaAtiva': 'Inativo', 9: 'Inativo', 10: 'Inativo'} Inativo False True True
{2022: {'ultimaAtiva': 'Encarregado', 3: 'Encarregado'}, 2023: {'ultimaAtiva': 'Inativo', 5: 'Inativo', 9: 'Inativo', 10: 'Inativo'}} {'ultimaAtiva': 'Inativo', 5: 'Inativo', 9: 'Inativo', 10: 'Inativo'} Inativo False True True
{2022: {'ultimaAtiva': 'Inativo', 3: 'Encarregado', 8: 'Inativo'}, 2023: {'ultimaAtiva': 'Inativo', 9: 'Inativo', 10: 'Inativo'}} {'ultimaAtiva': 'Inativo', 9: 'Inativo', 10: 'Inativo'} Inativo False True True
{2022: {'ultimaAtiva': 'Encarregado', 4: 'Encarregado'}, 2023: {'ultimaAtiva': 'Inativo', 9: 'Inativo', 10: 'Inativo'}} {'ultimaAt

{2022: {'ultimaAtiva': 'Coordenador', 4: 'Coordenador'}, 2023: {'ultimaAtiva': 'Coordenador', 7: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 7: 'Coordenador'} Coordenador True True True
{2022: {'ultimaAtiva': 'Coordenador', 4: 'Coordenador'}, 2023: {'ultimaAtiva': 'Coordenador', 7: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 7: 'Coordenador'} Coordenador True True True
{2022: {'ultimaAtiva': 'Coordenador', 4: 'Coordenador'}, 2023: {'ultimaAtiva': 'Coordenador', 7: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 7: 'Coordenador'} Coordenador True True True
{2022: {'ultimaAtiva': 'Inativo', 4: 'Coordenador', 6: 'Inativo'}} {'ultimaAtiva': 'Inativo', 4: 'Coordenador', 6: 'Inativo'} Inativo False True False
{2022: {'ultimaAtiva': 'Apontador', 4: 'Apontador'}, 2023: {'ultimaAtiva': 'Inativo', 3: 'Inativo'}} {'ultimaAtiva': 'Inativo', 3: 'Inativo'} Inativo False True True
{2022: {'ultimaAtiva': 'Apontador', 5: 'Apontador', 10: 'Inativo', 11: 'Apontador'}, 2023: {'ultimaAtiva': 'Inativo'

{2022: {'ultimaAtiva': 'Coordenador', 4: 'Coordenador'}, 2024: {'ultimaAtiva': 'Inativo', 1: 'Coordenador', 'Inativo mesmo mês': 1}} {'ultimaAtiva': 'Inativo', 1: 'Coordenador', 'Inativo mesmo mês': 1} Coordenador True False True
{2022: {'ultimaAtiva': 'Coordenador', 4: 'Coordenador'}, 2024: {'ultimaAtiva': 'Inativo', 1: 'Coordenador', 'Inativo mesmo mês': 1}} {'ultimaAtiva': 'Inativo', 1: 'Coordenador', 'Inativo mesmo mês': 1} Coordenador True False True
{2022: {'ultimaAtiva': 'Coordenador', 4: 'Coordenador'}, 2024: {'ultimaAtiva': 'Inativo', 1: 'Coordenador', 'Inativo mesmo mês': 1}} {'ultimaAtiva': 'Inativo', 1: 'Coordenador', 'Inativo mesmo mês': 1} Coordenador True False True
{2022: {'ultimaAtiva': 'Inativo', 5: 'Encarregado', 6: 'Inativo'}} {'ultimaAtiva': 'Inativo', 5: 'Encarregado', 6: 'Inativo'} Inativo False True False
{2022: {'ultimaAtiva': 'Encarregado', 5: 'Encarregado', 6: 'Encarregado'}, 2024: {'ultimaAtiva': 'Inativo', 1: 'Encarregado', 'Inativo mesmo mês': 1}} {'ultima

{2023: {'ultimaAtiva': 'Sala técnica', 7: 'Sala técnica'}, 2024: {'ultimaAtiva': 'Inativo', 1: 'Inativo'}} {'ultimaAtiva': 'Inativo', 1: 'Inativo'} Inativo False True True
{2023: {'ultimaAtiva': 'Inativo', 8: 'Sala técnica', 10: 'Inativo'}} {'ultimaAtiva': 'Inativo', 8: 'Sala técnica', 10: 'Inativo'} Inativo False True False
{2023: {'ultimaAtiva': 'Sala técnica', 8: 'Sala técnica', 'Inativo mesmo mês': 8, 9: 'Sala técnica'}} {'ultimaAtiva': 'Sala técnica', 8: 'Sala técnica', 'Inativo mesmo mês': 8, 9: 'Sala técnica'} Sala técnica True True False
{2023: {'ultimaAtiva': 'Usuário de campo', 8: 'Usuário de campo'}} {'ultimaAtiva': 'Usuário de campo', 8: 'Usuário de campo'} Usuário de campo True True False
{2023: {'ultimaAtiva': 'Inativo', 8: 'Usuário de campo', 'Inativo mesmo mês': 8}} {'ultimaAtiva': 'Inativo', 8: 'Usuário de campo', 'Inativo mesmo mês': 8} Usuário de campo True False False
{2023: {'ultimaAtiva': 'Usuário de campo', 8: 'Usuário de campo'}} {'ultimaAtiva': 'Usuário de camp

{2023: {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'} Coordenador True True False
{2023: {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'} Coordenador True True False
{2023: {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'}, 2024: {'ultimaAtiva': 'Inativo', 1: 'Inativo'}} {'ultimaAtiva': 'Inativo', 1: 'Inativo'} Inativo False True True
{2023: {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'} Coordenador True True False
{2023: {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'}} {'ultimaAtiva': 'Coordenador', 8: 'Coordenador'} Coordenador True True False
{2023: {'ultimaAtiva': 'Usuário de campo', 8: 'Usuário de campo', 10: 'Usuário de campo'}} {'ultimaAtiva': 'Usuário de campo', 8: 'Usuário de campo', 10: 'Usuário de campo'} Usuário de campo True True False
{2023: {'ultimaAtiva': 'Usuário de campo', 8: 'Usuário de campo'}, 2024: {'ultimaAtiva'

In [34]:
dfs[5]

,Usuários ativos
0,Regiani Machado


In [45]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name='Pavicolet (Amazonas)'
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).distinct('user').order_by('user')
historicoUsuarios.count()

33

In [46]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name='Pavicolet (Amapá)'
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).distinct('user').order_by('user')
historicoUsuarios.count()

35

In [79]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=companies
        ).only('company__company_group__name','history_date','user').exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).distinct('user').order_by('user')
historicoUsuarios.count()

379

In [80]:
historicoUsuarios

<QuerySet [<HistoricalUserInCompany: [b0c55603-e20e-4e98-aa83-2f7b220a0b57: Afirma Engenharia Viária] equipeafirma04 as of 2023-10-11 20:58:37.399872+00:00>, <HistoricalUserInCompany: [91d69c92-2284-4570-9011-760a5a74a708: Entrevias] marciel.souza as of 2021-01-08 14:57:21.797460+00:00>, <HistoricalUserInCompany: [8e70b23f-907a-4e9a-ae4e-fb316dda2831: Construtora Castilho] amanda.motta@castilho.com.br as of 2023-08-15 12:49:23.816402+00:00>, <HistoricalUserInCompany: [91d69c92-2284-4570-9011-760a5a74a708: Entrevias] sumaya.andrade as of 2023-05-24 12:13:22.016933+00:00>, <HistoricalUserInCompany: [91d69c92-2284-4570-9011-760a5a74a708: Entrevias] Josieldo.Deomondes as of 2023-03-16 15:56:43.165083+00:00>, <HistoricalUserInCompany: [a3fca710-54b8-4ee1-8375-1a908342438d: LCM Construção e Comércio] wellington.norato as of 2023-08-10 17:46:07.678311+00:00>, <HistoricalUserInCompany: [91d69c92-2284-4570-9011-760a5a74a708: Entrevias] leandro.donizete as of 2021-04-22 18:12:39.692392+00:00>, <

In [81]:
permissoes_history_user(historicoUsuarios)

ValueError: max() arg is an empty sequence

In [68]:
dados=[]
for i,a in enumerate(historicoUsuarios):
    try:
        if a.user.username not in dados:
            dados.append(a.user.username)
    except:
        pass
len(dados)

63

In [62]:
for i,a in enumerate(historicoUsuarios):
    print(a.user.username)

cristiano.oliveira
kledson.brito
Joaooliveira
aderbal.formiga
wal-liano.deka
wesllem.serudo
misael.serrao
lucas.wilian
hugo.isslle
luiz.arcanjo
edielson.campos
zaqueu.oliveira
Antônio
carlos.porto
geovane.almeida
alvaro.aurelio
vilmar.junior
vinicius.bortolini
cristiano.pantoja
reginaldo
gustavo.Monteiro
edgar.silva
alexandre.assef
antonio.felix
j.lima
souza.messias
aderbal.relvas
elizanio.vieira
wembleysantos
welington.diego
Eliton.Costa
fredson.venque
davi.silva
henrique.moussa
isenildo.torres
leonardo.martins
eldon.araujo
ruan.carlos
flavio.tavares
Wellington
ronald.souza
richard.maradona
tassio.janio
edmilson.peixoto
welington.bortolini
edvaldo.campos
Marcelo
matheus.eduardo
mateus.tavares
marcus.abreu
nelson.roriz
cleimilton.melo
saulo.sena
mailso.dias
erick.leite
neandro.braga


DoesNotExist: User matching query does not exist.

# Executar no caso da Entrevias

In [30]:
# historicoUsuarios=UserInCompany.history.model.objects.filter( Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr'), history_date__year= 2023,company__name=company).exclude(user__email__icontains='kartado').exclude(user__email__icontains='roadlabs').exclude(user__email__icontains='hermes').order_by( 'history_date')
# historicoUsuarios.count()

In [31]:
# historicoUsuarios3=UserInCompany.history.model.objects.filter(company__name__icontains=company, history_date__year= 2022).exclude(user__email__icontains='kartado').exclude(user__email__icontains='roadlabs').exclude(user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95').exclude(user__email__icontains='hermes').order_by( 'history_date')
historicoUsuarios=UserInCompany.history.model.objects.filter(Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr'),company__name__icontains=company,history_date__year=year).exclude(user__email__icontains='kartado').exclude(user__email__icontains='roadlabs').exclude(user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95').exclude(user__email__icontains='hermes').order_by( 'history_date')
historicoUsuarios.count()

0

In [61]:
result_list = historicoUsuarios

In [62]:
len(result_list)

52

In [7]:
df3 = pd.DataFrame(data=users_permissions)

In [8]:
df3

,0
0,Lia Pimentel
1,Gustavo Pizzo
2,André Reis
3,Bruno Fonseca
4,Vinicius Bessa


In [56]:
for a in permissoes_history_user(result_list)[0]:
    print(a)

Welington Bortolini Coordenador
Nelson Roriz Coordenador
Carlos Porto Coordenador
Vinícius Bortolini Inativo
Cristiano Oliveira Apontador
Abrão Baloeiro Dutra Apontador
Cristiano Pantoja Oliveira Inativo
Zaqueu de Lima Oliveira Inativo
Marcus Vinicius C Abreu Coordenador
Mailso Dias Apontador
Misael Serrão Apontador
geovane dos santos almeida Apontador
MESSIAS MORAES DE SOUZA Inativo
Jonas de Lima Freire Apontador
WELINGTON DIEGO COSTA Inativo
BENNERSON RIBEIRO Visualizador
HENRIQUE MOUSSA Visualizador
ELIZANIO VIEIRA Visualizador
Lucas Wilian de Almeida Santos Inativo
Diego de Jesus Amoras Apontador
Ronald Souza do Carmo Inativo
Eldon Araújo Marques Apontador
ADERBAL FORMIGA RELVAS Inativo
Edgar Silva Coordenador
Davi Silva Paixão Apontador
Saulo Daniel Apontador
Wellington Fagundes Sala técnica
Antônio Barbosa Inativo
Marcelo Martins Inativo
Wembley Santos Inativo
Jonnattan Gustavo Monteiro de Matos Inativo
JOÃO ECCEL Coordenador
Ruan Carlos Souza da Silva Inativo
Welington Bortolini

In [49]:
permissoesUsuarios = {}
for a in result_list:
    try:
    #print(a.user.get_full_name())
        if not permissoesUsuarios.get(a.user.get_full_name()):
            permissoesUsuarios[a.user.get_full_name()] = {}
    
        permissoesUsuarios[a.user.get_full_name()][a.history_date.month] = a.permissions.name
    except:
        pass

In [50]:
permissoesUsuarios

{'Welington Bortolini': {4: 'Coordenador'},
 'Nelson Roriz': {4: 'Coordenador'},
 'Carlos Porto': {4: 'Coordenador'},
 'Vinícius Bortolini': {4: 'Coordenador', 6: 'Inativo'},
 'Cristiano Oliveira': {4: 'Apontador', 3: 'Inativo'},
 'Abrão Baloeiro Dutra': {5: 'Apontador',
  10: 'Inativo',
  11: 'Apontador',
  3: 'Inativo'},
 'Cristiano Pantoja Oliveira': {6: 'Inativo', 3: 'Inativo'},
 'Zaqueu de Lima Oliveira': {6: 'Apontador',
  9: 'Encarregado com Configurações',
  8: 'Inativo'},
 'Marcus Vinicius C Abreu': {6: 'Coordenador', 1: 'Coordenador'},
 'Mailso Dias': {10: 'Apontador'},
 'Misael Serrão': {11: 'Apontador', 3: 'Inativo'},
 'geovane dos santos almeida': {12: 'Apontador', 3: 'Inativo'},
 'MESSIAS MORAES DE SOUZA': {1: 'Apontador', 7: 'Inativo'},
 'Jonas de Lima Freire': {2: 'Apontador'},
 'WELINGTON DIEGO COSTA': {3: 'Apontador', 8: 'Inativo'},
 'BENNERSON RIBEIRO': {3: 'Visualizador'},
 'HENRIQUE MOUSSA': {3: 'Visualizador'},
 'ELIZANIO VIEIRA': {3: 'Visualizador'},
 'Lucas Wili

In [51]:
users={}
for mes in range(1,12):
    try:
        for a in result_list:
            if a.history_date.month <= mes:
                if not users.get(mes):
                    users[mes] = set()
                users[mes].add(a.user.get_full_name())
    except:
        pass
users

{1: {'MESSIAS MORAES DE SOUZA', 'Marcus Vinicius C Abreu'},
 2: {'Jonas de Lima Freire',
  'MESSIAS MORAES DE SOUZA',
  'Marcus Vinicius C Abreu'},
 3: {'Jonas de Lima Freire',
  'MESSIAS MORAES DE SOUZA',
  'Marcus Vinicius C Abreu'},
 4: {'Carlos Porto',
  'Cristiano Oliveira',
  'Jonas de Lima Freire',
  'MESSIAS MORAES DE SOUZA',
  'Marcus Vinicius C Abreu',
  'Nelson Roriz',
  'Vinícius Bortolini',
  'Welington Bortolini'},
 5: {'Abrão Baloeiro Dutra',
  'Carlos Porto',
  'Cristiano Oliveira',
  'Jonas de Lima Freire',
  'MESSIAS MORAES DE SOUZA',
  'Marcus Vinicius C Abreu',
  'Nelson Roriz',
  'Vinícius Bortolini',
  'Welington Bortolini'},
 6: {'Abrão Baloeiro Dutra',
  'Carlos Porto',
  'Cristiano Oliveira',
  'Cristiano Pantoja Oliveira',
  'Jonas de Lima Freire',
  'MESSIAS MORAES DE SOUZA',
  'Marcus Vinicius C Abreu',
  'Nelson Roriz',
  'Vinícius Bortolini',
  'Welington Bortolini',
  'Zaqueu de Lima Oliveira'},
 7: {'Abrão Baloeiro Dutra',
  'Carlos Porto',
  'Cristiano 

In [52]:
permissoesUsuarios

{'Welington Bortolini': {4: 'Coordenador'},
 'Nelson Roriz': {4: 'Coordenador'},
 'Carlos Porto': {4: 'Coordenador'},
 'Vinícius Bortolini': {4: 'Coordenador', 6: 'Inativo'},
 'Cristiano Oliveira': {4: 'Apontador', 3: 'Inativo'},
 'Abrão Baloeiro Dutra': {5: 'Apontador',
  10: 'Inativo',
  11: 'Apontador',
  3: 'Inativo'},
 'Cristiano Pantoja Oliveira': {6: 'Inativo', 3: 'Inativo'},
 'Zaqueu de Lima Oliveira': {6: 'Apontador',
  9: 'Encarregado com Configurações',
  8: 'Inativo'},
 'Marcus Vinicius C Abreu': {6: 'Coordenador', 1: 'Coordenador'},
 'Mailso Dias': {10: 'Apontador'},
 'Misael Serrão': {11: 'Apontador', 3: 'Inativo'},
 'geovane dos santos almeida': {12: 'Apontador', 3: 'Inativo'},
 'MESSIAS MORAES DE SOUZA': {1: 'Apontador', 7: 'Inativo'},
 'Jonas de Lima Freire': {2: 'Apontador'},
 'WELINGTON DIEGO COSTA': {3: 'Apontador', 8: 'Inativo'},
 'BENNERSON RIBEIRO': {3: 'Visualizador'},
 'HENRIQUE MOUSSA': {3: 'Visualizador'},
 'ELIZANIO VIEIRA': {3: 'Visualizador'},
 'Lucas Wili

In [47]:
usuariosAtivos = []
usuariosInativos = []
for nome, mes in permissoesUsuarios.items():
    item = max(list(mes.keys()))
    cargo_atual = mes[item]
    print(mes,nome,item)
    if cargo_atual != 'Inativo':
        usuariosAtivos.append(nome)
    else:
        usuariosInativos.append(nome)


{4: 'Coordenador'} Welington Bortolini 4
{4: 'Coordenador'} Nelson Roriz 4
{4: 'Coordenador'} Carlos Porto 4
{4: 'Coordenador', 6: 'Inativo'} Vinícius Bortolini 6
{4: 'Apontador', 3: 'Inativo'} Cristiano Oliveira 4
{5: 'Apontador', 10: 'Inativo', 11: 'Apontador', 3: 'Inativo'} Abrão Baloeiro Dutra 11
{6: 'Inativo', 3: 'Inativo'} Cristiano Pantoja Oliveira 6
{6: 'Apontador', 9: 'Encarregado com Configurações', 8: 'Inativo'} Zaqueu de Lima Oliveira 9
{6: 'Coordenador', 1: 'Coordenador'} Marcus Vinicius C Abreu 6
{10: 'Apontador'} Mailso Dias 10
{11: 'Apontador', 3: 'Inativo'} Misael Serrão 11
{12: 'Apontador', 3: 'Inativo'} geovane dos santos almeida 12
{1: 'Apontador', 7: 'Inativo'} MESSIAS MORAES DE SOUZA 7
{2: 'Apontador'} Jonas de Lima Freire 2
{3: 'Apontador', 8: 'Inativo'} WELINGTON DIEGO COSTA 8
{3: 'Visualizador'} BENNERSON RIBEIRO 3
{3: 'Visualizador'} HENRIQUE MOUSSA 3
{3: 'Visualizador'} ELIZANIO VIEIRA 3
{3: 'Inativo', 8: 'Apontador', 10: 'Inativo'} Lucas Wilian de Almeida Sa

In [45]:
ui=UserInCompany.objects.get(user__username='zaqueu.oliveira')

In [57]:
for a in ui.history.all().order_by('-history_date'):
    print(a.user.username,a.permissions.name,a.history_date,a.history_user)

zaqueu.oliveira Inativo 2023-12-04 17:08:23.641413+00:00 rlcs - Kartado CS
zaqueu.oliveira Inativo 2023-08-14 15:40:12.978828+00:00 edgar.silva - Edgar Silva
zaqueu.oliveira Encarregado com Configurações 2022-09-29 14:00:20.072445+00:00 rlcs - Kartado CS
zaqueu.oliveira Apontador 2022-06-03 14:23:04.140809+00:00 marina.carpes - Marina Carpes


In [19]:
usuariosAtivos,usuariosInativos

(['Welington Bortolini',
  'Nelson Roriz',
  'Carlos Porto',
  'Cristiano Oliveira',
  'Abrão Baloeiro Dutra',
  'Zaqueu de Lima Oliveira',
  'Marcus Vinicius C Abreu',
  'Mailso Dias',
  'Misael Serrão',
  'geovane dos santos almeida',
  'Jonas de Lima Freire',
  'BENNERSON RIBEIRO',
  'HENRIQUE MOUSSA',
  'ELIZANIO VIEIRA',
  'Diego de Jesus Amoras',
  'Eldon Araújo Marques',
  'Edgar Silva',
  'Davi Silva Paixão',
  'Saulo Daniel',
  'Wellington Fagundes',
  'JOÃO ECCEL'],
 ['Vinícius Bortolini',
  'Cristiano Pantoja Oliveira',
  'MESSIAS MORAES DE SOUZA',
  'WELINGTON DIEGO COSTA',
  'Lucas Wilian de Almeida Santos',
  'Ronald Souza do Carmo',
  'ADERBAL FORMIGA RELVAS',
  'Antônio Barbosa',
  'Marcelo Martins',
  'Wembley Santos',
  'Jonnattan Gustavo Monteiro de Matos',
  'Ruan Carlos Souza da Silva'])

In [20]:
for i in usuariosAtivos:
    print(i)

Welington Bortolini
Nelson Roriz
Carlos Porto
Cristiano Oliveira
Abrão Baloeiro Dutra
Zaqueu de Lima Oliveira
Marcus Vinicius C Abreu
Mailso Dias
Misael Serrão
geovane dos santos almeida
Jonas de Lima Freire
BENNERSON RIBEIRO
HENRIQUE MOUSSA
ELIZANIO VIEIRA
Diego de Jesus Amoras
Eldon Araújo Marques
Edgar Silva
Davi Silva Paixão
Saulo Daniel
Wellington Fagundes
JOÃO ECCEL


In [21]:
df = pd.DataFrame(usuariosAtivos)

In [22]:
df

,0
0,Welington Bortolini
1,Nelson Roriz
2,Carlos Porto
3,Cristiano Oliveira
4,Abrão Baloeiro Dutra
5,Zaqueu de Lima Oliveira
6,Marcus Vinicius C Abreu
7,Mailso Dias
8,Misael Serrão
9,geovane dos santos almeida


# Usuários para Financeiro Meses anteriores

In [59]:
ultimo_mes=9

In [60]:
filename='Usuários Ativos Financeiro Meses Anteriores'
pd.DataFrame().to_excel(filename+'.xlsx')

In [61]:
for company in companies:
    if company is not 'Entrevias':
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name=company
            ,history_date__month__in=[i for i in reversed(range(1,ultimo_mes+1))],
            history_date__year=year
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    else:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr'),
            company__name__icontains=company
            ,history_date__month__in=[i for i in reversed(range(1,ultimo_mes+1))],
            history_date__year=year
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    
    users_permissions = permissoes_history_user(historicoUsuarios)[0] if len(permissoes_history_user(historicoUsuarios)[0]) > 0 else 0
    
    if users_permissions != 0:
        df2 = pd.DataFrame(data=users_permissions,columns=['Usuários ativos'])
        with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=company,na_rep="-")

In [40]:
df2

,Usuários ativos
0,José Rafael Tavares
1,Eudes Teles de Souza
2,Samuel Viana dos Santos
3,Denis Alves Silva
4,Marcos Cipriano
5,Anderson Adauto
6,Jessé Campos da Silva
7,Thiago Oliveira


In [43]:
company=Company.objects.get(name='Eixo Sp - Pavimento')

In [44]:
company

<Company: 8174e68f-3a8f-4013-98dd-062954e4c0da: Eixo Sp - Pavimento>

In [48]:
import pandas as pd

In [49]:
df = pd.DataFrame(data={'name':'Luan'})

ValueError: If using all scalar values, you must pass an index

In [50]:
df

NameError: name 'df' is not defined

# Teste

In [4]:
def permissoes_history_user(result_list):
    permissoesUsuarios = {}
    for a in result_list.order_by('user','history_date'):
        try:
#             print(a.permissions.name)
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if not permissoesUsuarios.get(a.user.get_full_name()).get(a.history_date.year): 
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year] = {}
            if not permissoesUsuarios.get(a.user.get_full_name()).get(a.history_date.year).get(a.history_date.month):
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.history_date.month] = {}
            if permissoesUsuarios.get(
                a.user.get_full_name()
            ).get(
                a.history_date.year
            ).get(
                a.history_date.month
            ).get(a.history_date.day):
                perm_do_dia = permissoesUsuarios.get(
                    a.user.get_full_name()
                ).get(
                    a.history_date.year
                ).get(
                    a.history_date.month
                ).get(
                    a.history_date.day
                )
                if not isinstance(permissoesUsuarios.get(a.user.get_full_name()).get(a.history_date.year).get(a.history_date.month).get(a.history_date.day),list):
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.history_date.month][a.history_date.day]=[perm_do_dia,a.permissions.name]
                else:
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.history_date.month][a.history_date.day].append(a.permissions.name)
            else:
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.history_date.month][a.history_date.day] = a.permissions.name
#             print(permissoesUsuarios)
        except:
            pass

    usuariosAtivos = []
    usuariosInativos = []
#     print(permissoesUsuarios)
    for nome, ano in permissoesUsuarios.items():
        print(nome, ano)
        ultimo_ano = sorted(list(ano.keys()))[-1]
        ano_anterior = sorted(list(ano.keys()))[-2] if len(sorted(list(ano.keys()))) > 1 else ultimo_ano
        ultimo_mes = max(list(ano[ultimo_ano].keys()))
        ultimo_dia = max(list(ano[ultimo_ano][ultimo_mes].keys()))
        penultimo_mes = ultimo_mes
        penultimo_mes_ult_dia = ultimo_dia
        if ano_anterior != ultimo_ano:
            penultimo_mes = max(list(ano[ano_anterior].keys()))
            penultimo_mes_ult_dia = max(list(ano[ano_anterior][penultimo_mes].keys()))
        ultimo_mes = max(list(ano[ultimo_ano].keys()))
        ultimo_dia = max(list(ano[ultimo_ano][ultimo_mes].keys()))
        cargo = ano[ultimo_ano][ultimo_mes][ultimo_dia]
        if not isinstance(cargo,list):
            cargo_atual = cargo
            cargo_anterior = ano.get(ano_anterior).get(penultimo_mes).get(penultimo_mes_ult_dia)
#             cargo_cobrar = cargo_anterior if cargo_anterior is not None and cargo_anterior != 'Inativo' and ano_anterior
        else:
            cargo_atual = cargo[-1]
            cargo_anterior = cargo[-2]
#         item_mes = max([x for x in cargo_atual.keys() if isinstance(x,int)])
        print(cargo_atual,cargo_anterior,ano_anterior,penultimo_mes,penultimo_mes_ult_dia)
#         if ultimo_ano != ano_anterior
#         if (cargo_atual[item_mes] != 'Inativo' and cargo_atual.get('ultimaAtiva') != 'Inativo') or cargo_atual.get('Inativo mesmo mês') != item_mes or item_min_ano < item_ano:
#             usuariosAtivos.append(nome)
#         else:
#             usuariosInativos.append(nome)

    return usuariosAtivos,usuariosInativos

In [11]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__icontains='Construtora Castilho'
            ,user__username='lteste'
        ).order_by('-history_date')
historicoUsuarios.count()

3

In [12]:
for a in historicoUsuarios:
    print(a.permissions.name,a.history_date)

Coordenador 2024-01-31 19:38:03.314893+00:00
Inativo(muitos) 2024-01-31 17:09:52.945318+00:00
Suporte 2024-01-31 17:09:30.063547+00:00


In [7]:
permissoes_history_user(historicoUsuarios)

Luan Poletti {2023: {8: {29: 'Coordenador'}, 10: {27: 'Usuário de campo'}}, 2024: {1: {31: ['Suporte', 'Inativo(muitos)']}, 2: {1: 'Inativo(muitos)'}}}
Inativo(muitos) Usuário de campo 2023 10 27


([], [])

In [179]:
dicio={'Luan Poletti': {2023: {8: 'Coordenador', 10: 'Usuário de campo'}, 2024: {1: 'Suporte'}}}

In [156]:
sorted(list(dicio['Luan Poletti'][2023].keys()))[-2]

8

In [193]:
type(a.history_date.month)

int

In [182]:
dicio.get(2021)

In [8]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name='Construtora Castilho'
            ,history_date__lt='2024-2-1'
        ).values('history_date__month',
                 'permissions__name',
                 'history_date',
                 'user__first_name',
                'user__last_name','user__username').order_by('-history_date')
historicoUsuarios.count()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField HistoricalUserInCompany.history_date received a naive datetime (2024-02-01 00:00:00) while time zone support is active.
  RuntimeWarning)


84

In [248]:
usuariosAtivos=[]
usuariosInativos=[]
for a in historicoUsuarios:
#     if 'Zaqueu' in a.get('user__first_name'):
#         print(a['user__username'])
    fname=a['user__first_name'] if a['user__first_name'] is not None else '' 
    lname=a['user__last_name'] if a['user__last_name'] is not None else ''
    fullname = fname+' '+lname
    if fullname not in usuariosAtivos or fullname in usuariosInativos:
        if a['permissions__name'] != "Inativo" and a['history_date'].day > 1 and fullname not in usuariosAtivos:
            usuariosAtivos.append(fullname)
        elif fullname not in usuariosAtivos and a['permissions__name'] == "Inativo":
            usuariosInativos.append(fullname)
    

In [249]:
len(usuariosAtivos),len(usuariosInativos),usuariosAtivos,usuariosInativos

(50,
 0,
 ['teste 2 testande',
  'Teste joaonitzke',
  'Luan Poletti',
  'Renan Pereira',
  'Romulo Lopes',
  'Gustavo Amorim',
  'Camila Prado',
  'Klecio Barbosa',
  'Brunno Dias',
  'Adriano Macedo',
  'Edevaldo Diniz',
  'Leonardo Alcantara',
  'Giovanni valentim',
  'Rafael Mendes',
  'Carlos Gomes',
  'Axcel Cezar',
  'Leonardo Minski',
  'Evandro Ferreira',
  'João Nitzke',
  'Luiza Franco',
  'Rafael Guimarães',
  'Rodrigo Cunha',
  'Douglas Campos',
  'Antonio Carlos Lindoso',
  'Walison Salvador',
  'Mauro Campos',
  'Alex Fonseca',
  'Vinicius Melo',
  'Talita Arantes',
  'Monique Marcelino',
  'Carlos Daniel',
  'Mauricio Leite',
  'Karla Zorzam Brito',
  'Kartado CS',
  'Eder de Souza',
  'Carlos de Paula',
  'Celso Vechio',
  'Joelton Alves',
  'Mateus Volpe',
  'Daniel Pereira',
  'Gabriel Rodrigues',
  'Glaucio Moraes',
  'Daniella Da Silva Gomes',
  'Leonardo Carvalho',
  'Amanda Motta2',
  'Alvaro Neto',
  'Amanda Motta',
  'Pedro Fornari',
  'Pedro Cardoso',
  'Victo

In [278]:
Company.objects.filter(name__icontains='Castilho')

<QuerySet [<Company: 8e70b23f-907a-4e9a-ae4e-fb316dda2831: Construtora Castilho>, <Company: 3be541f1-3489-4ec5-b44a-cd1faab091e2: Construtora Castilho (Ferrovias)>, <Company: 837181f0-5c09-42a6-910d-e799607b8701: Construtora Castilho (Prefeitura Curitiba)>, <Company: 3d5f2e4d-f106-4415-973a-1b0e2ba2ca3b: Castilho Umuarama>]>

In [83]:
for a in Company.objects.filter(name='Construtora Castilho'):
    print(a.name,a.company_group)

Construtora Castilho [rlcs] Castilho


In [250]:
def permissoes_history_user_all(result_list):
    permissoesUsuarios = {}
    for a in result_list.order_by('history_date'):
        try:
        #print(a.user.get_full_name())
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if not permissoesUsuarios.get(a.user.get_full_name()).get(a.history_date.year): 
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year] = {}
            if a.history_date.month in permissoesUsuarios.get(a.user.get_full_name()).get(a.history_date.year).keys():
                if a.permissions.name == "Inativo":
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.permissions.name+' mesmo mês'] = a.history_date.month
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year]['ultimaAtiva'] = a.permissions.name
                else:
                    permissoesUsuarios[a.user.get_full_name()][a.history_date.year]['ultimaAtiva'] = a.permissions.name
            else:
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year]['ultimaAtiva'] = a.permissions.name
                permissoesUsuarios[a.user.get_full_name()][a.history_date.year][a.history_date.month] = a.permissions.name
#             print(permissoesUsuarios)
        except:
            pass
    reformed_dict = {}
    for outerKey, innerDict in permissoesUsuarios.items():
        for innerKey, values in innerDict.items():
            reformed_dict[(outerKey,
                           innerKey)] = values
#     print(reformed_dict)
    df = pd.DataFrame(data=reformed_dict)
    return df

In [251]:
 historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__icontains='Pavicolet'
 ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        )
historicoUsuarios.count()

153

In [241]:
for a in historicoUsuarios:
    print(a.user.get_full_name(),a.user.username,'|',a.permissions.name,'|',a.history_date)

João Fachini joao.fachini | Suporte | 2023-12-11 18:47:14.494663+00:00
João Fachini joao.fachini | Administrador | 2023-12-11 18:43:55.059721+00:00
João Fachini joao.fachini | Administrador | 2023-12-11 18:37:20.975177+00:00
João Fachini joao.fachini | Suporte | 2023-12-11 18:26:37.882987+00:00
João Fachini joao.fachini | Administrador | 2023-12-08 14:19:46.382117+00:00
Delbo Rodrigues delbo.rodrigues | Usuário de campo | 2023-12-08 14:14:03.211749+00:00
João Fachini joao.fachini | Suporte | 2023-12-07 20:08:14.451193+00:00
Marcos Fachini marcos.fachini | Administrador | 2023-12-07 11:19:18.621316+00:00
Luiz Antônio luiz.antonio | Coordenador de Equipe | 2023-12-07 11:18:56.017264+00:00
Flávio Soares flavio.soares | Coordenador de Equipe | 2023-12-07 11:18:15.070080+00:00
Celso dos Santos celso.santos | Coordenador de Equipe | 2023-12-06 17:30:51.652157+00:00
Delbo Rodrigues delbo.rodrigues | Coordenador de Equipe | 2023-11-27 14:23:23.605448+00:00
Cleberson Rocha cleberson.rocha | Coo

DoesNotExist: UserPermission matching query does not exist.

In [255]:
import pandas as pd
df=permissoes_history_user_all(historicoUsuarios)
df.to_excel('pavicolet.xlsx')
df

Welington Bortolini              Nelson Roriz               \
                           2022         2023         2022         2023   
ultimaAtiva         Coordenador  Coordenador  Coordenador  Coordenador   
4                   Coordenador          NaN  Coordenador          NaN   
7                           NaN  Coordenador          NaN  Coordenador   
6                           NaN          NaN          NaN          NaN   
2                           NaN          NaN          NaN          NaN   
3                           NaN          NaN          NaN          NaN   
5                           NaN          NaN          NaN          NaN   
10                          NaN          NaN          NaN          NaN   
11                          NaN          NaN          NaN          NaN   
9                           NaN          NaN          NaN          NaN   
8                           NaN          NaN          NaN          NaN   
12                          NaN          NaN          NaN          NaN   
1                           NaN          NaN          NaN          NaN   

            Carlos Porto              Vinícius Bortolini                   \
                    2022         2023               2022             2024   
ultimaAtiva  Coordenador  Coordenador    Inativo(muitos)  Inativo(muitos)   
4            Coordenador          NaN        Coordenador              NaN   
7                    NaN  Coordenador                NaN              NaN   
6                    NaN          NaN    Inativo(muitos)              NaN   
2                    NaN          NaN                NaN  Inativo(muitos)   
3                    NaN          NaN                NaN              NaN   
5                    NaN          NaN                NaN              NaN   
10                   NaN          NaN                NaN              NaN   
11                   NaN          NaN                NaN              NaN   
9                    NaN          NaN                NaN              NaN   
8                    NaN          NaN                NaN              NaN   
12                   NaN          NaN                NaN              NaN   
1                    NaN          NaN                NaN              NaN   

            Cristiano Oliveira                   ...   Jackson Soares  \
                          2022             2023  ...             2024   
ultimaAtiva          Apontador  Inativo(muitos)  ...  Inativo(muitos)   
4                    Apontador              NaN  ...              NaN   
7                          NaN              NaN  ...              NaN   
6                          NaN              NaN  ...              NaN   
2                          NaN              NaN  ...  Inativo(muitos)   
3                          NaN  Inativo(muitos)  ...              NaN   
5                          NaN              NaN  ...              NaN   
10                         NaN              NaN  ...              NaN   
11                         NaN              NaN  ...              NaN   
9                          NaN              NaN  ...              NaN   
8                          NaN              NaN  ...              NaN   
12                         NaN              NaN  ...              NaN   
1                          NaN              NaN  ...              NaN   

              Cleiton Marques  richard maradona  Edmilson Peixoto  \
                         2023              2023              2023   
ultimaAtiva  Usuário de campo  Usuário de campo  Usuário de campo   
4                         NaN               NaN               NaN   
7                         NaN               NaN               NaN   
6                         NaN               NaN               NaN   
2                         NaN               NaN               NaN   
3                         NaN               NaN               NaN   
5                         NaN               NaN               NaN   
10                        Na

In [254]:
df.loc['ultimaAtiva']

Welington Bortolini  2022         Coordenador
                     2023         Coordenador
Nelson Roriz         2022         Coordenador
                     2023         Coordenador
Carlos Porto         2022         Coordenador
                                   ...       
Joao Oliveira        2023     Inativo(muitos)
                     2024     Inativo(muitos)
reginaldo reg        2023    Usuário de campo
Antonio Costa        2023    Usuário de campo
Eliton costa         2024           Apontador
Name: ultimaAtiva, Length: 102, dtype: object

In [13]:
DailyReportOccurrence._meta.fields

(<django.db.models.fields.UUIDField: uuid>,
 <django.db.models.fields.related.ForeignKey: created_by>,
 <django.db.models.fields.related.ForeignKey: firm>,
 <django.db.models.fields.CharField: origin>,
 <django.db.models.fields.TimeField: starts_at>,
 <django.db.models.fields.TimeField: ends_at>,
 <django.db.models.fields.TimeField: impact_duration>,
 <django.db.models.fields.TextField: description>,
 <django.db.models.fields.TextField: extra_info>)